In [39]:
import logging.config
from logging import Logger, getLogger
from typing import List

from pytket.extensions.qulacs import QulacsBackend as TketQulacsBackend

from qlbm.components import (
    CQLBM,
)
from qlbm.infra import CircuitCompiler
from qlbm.lattice import CollisionlessLattice
from qlbm.tools.utils import create_directory_and_parents


In [50]:
from qlbm.tools.utils import get_circuit_properties


def benchmark(
    lattice_files: List[str],
    logger: Logger,
    dummy_logger: Logger,
    compiler_platform: List[str],
    target_platform: List[str],
    optimization_levels: List[int],
    backend: TketQulacsBackend | None,
    num_repetitions: int = 5,
) -> None:
    for rep in range(num_repetitions):
        logger.info(f"Repetition #{rep + 1} of {num_repetitions}")
        for count, lattice_file in enumerate(lattice_files):
            for opt_count, optimization_level in enumerate(optimization_levels):
                logger.info(
                    f"Combination #{(count * len(optimization_levels)) + opt_count + 1} of {len(lattice_files)*len(optimization_levels)}"
                )

                lattice_name = (
                    lattice_file.split("/")[-1].split(".")[0].replace("_", "-")
                )
                lattice = CollisionlessLattice(lattice_file, logger=dummy_logger)
                logger.info(
                    f"Lattice: {lattice_name}; opt={optimization_level}; num_qubits = {lattice.num_total_qubits};"
                )

                algorithm = CQLBM(lattice, logger=dummy_logger)
                logger.info(
                    f"Original circuit has properties: {get_circuit_properties(algorithm.circuit)}"
                )
                compiler = CircuitCompiler(
                    compiler_platform, target_platform, logger=logger
                )

                compiler.compile(
                    compile_object=algorithm,
                    backend=backend,
                    optimization_level=optimization_level,
                )

In [51]:
NUM_SHOTS = 2**14
NUM_STEPS = 5
ROOT_OUTPUT_DIR = "qlbm-output/benchmark-algorithm-scalability"

create_directory_and_parents(ROOT_OUTPUT_DIR)

In [52]:
!mkdir -p qlbm-output/benchmark-algorithm-scalability && touch qlbm-output/benchmark-algorithm-scalability/qlbm.log
!:> qlbm-output/benchmark-algorithm-scalability/qlbm.log

In [53]:
lattice_files = [
    "../lattices/2d_8x8_0_obstacle.json",
    "../lattices/2d_8x8_1_obstacle.json",
    "../lattices/2d_8x8_2_obstacle.json",
]

dummy_logger = getLogger("dummy")
# By logging at this point we ignore the output of circuit creation
logging.config.fileConfig("algorithm_scalability_logging.conf")
logger = getLogger("qlbm")

In [54]:
logger.info("Session: QISKIT")
benchmark(
    lattice_files,
    logger,
    dummy_logger,
    "QISKIT",
    "QULACS",
    [0],
    None,
    num_repetitions=1,
)

/home/calin/phd/qlbm/.venv/lib64/python3.11/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for unitary can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(
/home/calin/phd/qlbm/.venv/lib64/python3.11/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for unitary can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(
/home/calin/phd/qlbm/.venv/lib64/python3.11/site-packages/qiskit/providers/backend_compat.py:126: RuntimeWarning: No gate definition for unitary can be found and is being excluded from the generated target. You can use `custom_name_mapping` to provide a definition for this operation.
  warnings.warn(


In [55]:
logger.info("Session: TKET")
benchmark(
    lattice_files,
    logger,
    dummy_logger,
    "TKET",
    "QULACS",
    [0],
    TketQulacsBackend(),
    num_repetitions=1,
)


In [56]:
log_file = "qlbm-output/benchmark-algorithm-scalability/qlbm.log"
with open(log_file, "r") as f:
    lines = f.readlines()

session_line = [c for c, line in enumerate(lines) if "Session" in line][1]

lines_statevector_true = lines[:session_line]
lines_statevector_false = lines[session_line:]

In [ ]:
# Process statevector=True lines
combination_lines_indices = [
    c for c, line in enumerate(lines_statevector_true) if "Combination #" in line
]
session_line_indices = [
    c for c, line in enumerate(lines_statevector_true) if "Session" in line
]

sections = []
for c in range(len((combination_lines_indices))):
    if c < len(combination_lines_indices) - 1:
        sections.append(
            lines_statevector_true[
                combination_lines_indices[c] : combination_lines_indices[c + 1]
            ]
        )
    else:
        sections.append(lines_statevector_true[combination_lines_indices[c] :])

records = []
for c, section in enumerate(sections):
    section_info = section[1].split(": ")[-1].split("' ")
    lattice_name = section_info[0]
    compiler_platform = section_info[1].split("=")[-1]
    opt_level = section_info[2].split("=")[-1]
    num_qubits = section_info[3].split("=")[-1]

    original_props = (
        [line for line in section if "Original circuit" in line][0]
        .split("(")[-1]
        .rstrip(")")
        .split(", ")
    )
    compiled_props = (
        [line for line in section if "Compiled circuit" in line][0]
        .split("(")[-1]
        .rstrip(")")
        .split(", ")
    )
    duration = int(section[-1].split()[-2])

    records.append(
        {
            "Lattice": lattice_name[1],
            "Dimensions": lattice_name[0].split("-")[1],
            "Obstacles": int(lattice_name.split("-")[2]),
            "Circuit Qubits": num_qubits,
            "Initial Depth": 0,
            "Initial Gate No.": 0,
            "Compiled Depth": 0,
            "Compiled Gate No.": 0,
            "Duration (ns)": int(duration),
        }
    )

sv_true_df = pd.DataFrame.from_records(statevector_true_records)
sv_true_df